In [1]:
import pandas as pd
import numpy as np
import geoip2.database
import urllib3
from netaddr import *

# Nepieciesamie Python moduli

In [2]:
# Nolasam excel failu, saglabajam nolasito ka pandas.DataFrame objektu
# Nolasisana aiznem 2min 18s. Ir ok prieks 1e6 ierakstiem, 44mb faila un veca laptopa.
data = pd.read_excel("web_data.xlsx")

# Izvadam pirmos ierakstus
# Parbaudam ka izdevies nolasit, vai nav nepieciesams pielagot pd.read_excel metodes parametrus
data.head()

,user_id,context_ip,original_timestamp,url
0,01dc0670-f180-4646-92a8-5df7003a4e74,178.42.28.14,2017-06-30 23:59:52.911,https://www.sonarworks.com/speakers?utm_source...
1,e509dab0-8a2c-45e4-9a2a-41b8e1365639,74.87.39.188,2017-06-30 23:59:48.348,https://storeus.sonarworks.com/cart?uuid=e509d...
2,6491b4d2-9e0f-4f80-88a4-df272d41d075,98.247.18.133,2017-06-30 23:59:46.790,https://storeus.sonarworks.com/collections/son...
3,6262ec06-f8ac-4b37-b0f9-eb7077fdf7dc,69.142.145.113,2017-06-30 23:59:45.568,https://checkout.shopify.com/7492567/checkouts...
4,9d19fd2c-2adb-4b2b-b5db-56eceab40f5a,90.30.31.226,2017-06-30 23:59:43.871,https://www.sonarworks.com/install/systemwide?...


In [3]:
# Viss kartiba
# Iegusim informaciju par ieguto DataFrame objektu

data.describe()

,user_id,context_ip,original_timestamp,url
count,965886,965886,965886,965886
unique,222669,169628,965561,119748
top,f03ffd96-a320-4c00-8871-2a20b1a77eb5,83.243.92.42,2017-01-12 00:00:00.070000,http://www.sonarworks.com/
freq,3133,15841,14,83575
first,NaN,NaN,2017-01-01 00:00:00.070000,NaN
last,NaN,NaN,2017-06-30 23:59:52.911000,NaN


In [4]:
# Iegusim jaunu datu rami - df kas satures tikai unikalus context_ip. DataFrame ir nemainams(immutable) objekts.
# drop_duplicates metode -> Atrodam context_ip kopijas, paturam pedejo no kopijam df objekta.
# Hronologiski, 1. reize, kad unikals lietotajs veidoja savienojumu ar web lapu
# Aiznem 0.8 sekundes

df = data.drop_duplicates(subset='context_ip', keep='last')
df.describe()

,user_id,context_ip,original_timestamp,url
count,169628,169628,169628,169628
unique,143285,169628,169594,49877
top,38fd5196-71a3-44e3-a56a-4c773f89949c,45.37.229.51,2017-01-15 00:00:00.070000,http://www.sonarworks.com/
freq,169,1,5,27928
first,NaN,NaN,2017-01-01 00:00:00.070000,NaN
last,NaN,NaN,2017-06-30 23:59:52.911000,NaN


In [5]:
# No 965886 ierakstiem 169628 adreses ir unikalas
# Sanity check: Tas nozime, ka videji, katrs unikals lietotajs apmekle ~5 saites weblapa
# 6 menesu perioda ~170k unikali apmekletaji, ~28k unikalu lietotaju menesi
# Skiet racionali. Ejam talak

df.head()

,user_id,context_ip,original_timestamp,url
0,01dc0670-f180-4646-92a8-5df7003a4e74,178.42.28.14,2017-06-30 23:59:52.911,https://www.sonarworks.com/speakers?utm_source...
9,6491b4d2-9e0f-4f80-88a4-df272d41d075,98.247.18.133,2017-06-30 23:58:24.951,https://storeus.sonarworks.com/collections/hea...
19,bb43b475-2f06-43b1-ac1e-37543e689083,24.253.40.76,2017-06-30 23:54:38.123,https://www.sonarworks.com/
34,ac19e1c6-6aaa-47fa-9fd4-c8d106a1e0e6,46.39.54.3,2017-06-30 23:52:14.948,https://www.sonarworks.com/speakers?utm_source...
38,d8e3a64a-0db4-43a7-904d-63065210589f,83.184.238.8,2017-06-30 23:52:00.812,https://www.sonarworks.com/speakers?utm_source...


In [6]:
''' 
    Bridinajums: Nedaudz tehniski un sviestaini par to ka izvelejos
    parverst ip adreses. Varbut labak izlaist so teksta gabalu.
                                        
    Paslaik, prata nak 3 veidi ka veikt ip address -> country parversanu
    
    1: Veicot pieparsijumus web lapai.
    Ar urllib3 palidzibu veikt 'GET' pieprasijumu, piemeram, lapai, 
    https://tools.keycdn.com/geo?host=128.29.33.44. Ar BeautifulSoup4 palidzibu apstradat atbildi, izvelamies 
    lapas dalu, kur class = "panel panel-default", dala atrodam <tb>(tabulu),
    tabula atrodam 1. <tr> tagu(1. rinda tabula), no 1. rindas izvelkam ip adresi.Atkartojam to visam ip adresem
    
    Neizvelejos so metodi, jo 1)laikietilpigi 
                              2)web lapa var nebut prieciga par ~200k pieprasijumiem isa laika perioda
                              3)nepieciesama individuala pieeja katrai lapai, jarokas html/css/js koda
    
    2: Izmantojot datubazi, kas atrodas google drive.
    Problematiski ir salidzinat ipv4 addresses punktotaja formata(128.29.33.44), jo tie ir string tipa
    objekti, kas tiktu salidzinati leksikografisi, kas var radit problemas ar noteiktiem ip adresu diapazoniem.
    Naktos visas ip adreses ar funkcijas palidzibu parverst uz integer tipa 
    objektiem, piemeram, 178.42.28.14 ==> 2989104142, un tad
    parverst ip adresu datubazes kolonas Start,End ierakstus ar DataFrame metodi applymap() vai map(). 
    Pec tam salidzinat:
                        if (context_ip > start & context_ip < end)
                        True --> return country
                        False --> skaties cita diapazona
                        Else --> return 'Unknown'
                    
    Izdaram to visiem tabulas ierakstiem.
    
    3: Izmantojot moduli geoip2 un GeoLite2 bezmaksas datubazi(https://dev.maxmind.com/geoip/geoip2/geolite2/).
    Izvelos izmantot Python moduli/API geoip2(https://github.com/maxmind/GeoIP2-python), jo
    man tas skiet visvienkarsakais variants
'''
;

''

In [7]:
# Saksim iegut ip adresu attiecigas valstis
# Parbaudam vai ip adresu datubaze strada
reader = geoip2.database.Reader('/home/kursis/sonarworks/GeoLite2-Country.mmdb')
r = reader.country('178.42.28.14')
r.country.name

# Funkcija, kas ievaditajai ip adresei atgriezis attiecigo valsti.
# nepieciesama DataFrame objekta metodei map()
def get_country(addr):
    r = reader.country(addr)
    return r.country.name

In [8]:
# Nepieciesams izfiltret ierakstus no privata tikla un rezervetam adresem, jo tas neatrodas GeoLite2 datubaze
# source --> https://en.wikipedia.org/wiki/Reserved_IP_addresses

reserved = IPSet(['10.0.0.0/8', '172.16.0.0/12', '192.168.0.0/16','240.0.0.0/4','224.0.0.0/4',
                 '::ffff:0:0/96'])

# pagaidam pietiekami, velak var pielikt vel citus diapazonus

# Funkcija, kas nosaka vai ievadita adrese ir rezerveta/privata, atgriez True, ja rezerveta, False, ja publiska
# nepieciesama DataFrame objekta metodei map()
def is_reserved(addr):
    return addr in reserved

In [9]:
# Pievienojam datu ramim - df bool tipa kolonu ar vardu 'is_reserved'
# Tas palidzes izfiltret rezervetas/privatas adreses
# metode DataFrame.map(func) -> Katram kolonas elementam izpilda funkciju - func
# Aiznem 24s
df['is_reserved'] = df['context_ip'].map(is_reserved)
;

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


''

In [10]:
# Unikalu adresu skaits pirms filtracijas
df.describe()

,user_id,context_ip,original_timestamp,url,is_reserved
count,169628,169628,169628,169628,169628
unique,143285,169628,169594,49877,2
top,38fd5196-71a3-44e3-a56a-4c773f89949c,45.37.229.51,2017-01-15 00:00:00.070000,http://www.sonarworks.com/,False
freq,169,1,5,27928,169552
first,NaN,NaN,2017-01-01 00:00:00.070000,NaN,NaN
last,NaN,NaN,2017-06-30 23:59:52.911000,NaN,NaN


In [11]:
# Veidojam jaunu DataFrame objektu - df, kas satures tikai publiskas ip adreses
# Aiznem 68ms
df = df[df['is_reserved'] == False]

In [12]:
# Unikalu adresu skaits pec filtracijas(par ~70 mazak).
# Aizdomigi? 6 menesu laika tikai ~70 piekluves. Laikam, atkarigs no ta kada tikla atrodas web serveris.
df.describe()

,user_id,context_ip,original_timestamp,url,is_reserved
count,169552,169552,169552,169552,169552
unique,143243,169552,169521,49867,1
top,38fd5196-71a3-44e3-a56a-4c773f89949c,45.37.229.51,2017-01-15 00:00:00.070000,http://www.sonarworks.com/,False
freq,154,1,5,27911,169552
first,NaN,NaN,2017-01-01 00:00:00.070000,NaN,NaN
last,NaN,NaN,2017-06-30 23:59:52.911000,NaN,NaN


In [13]:
# Pievienosim datu ramim df, jaunu kolonu - country, kas satures valsts nosaukumus
# Aiznem 1m 55s
df['country'] = df['context_ip'].map(get_country)

In [14]:
# Apskatisim rezultatus
df.head()

,user_id,context_ip,original_timestamp,url,is_reserved,country
0,01dc0670-f180-4646-92a8-5df7003a4e74,178.42.28.14,2017-06-30 23:59:52.911,https://www.sonarworks.com/speakers?utm_source...,False,Poland
9,6491b4d2-9e0f-4f80-88a4-df272d41d075,98.247.18.133,2017-06-30 23:58:24.951,https://storeus.sonarworks.com/collections/hea...,False,United States
19,bb43b475-2f06-43b1-ac1e-37543e689083,24.253.40.76,2017-06-30 23:54:38.123,https://www.sonarworks.com/,False,United States
34,ac19e1c6-6aaa-47fa-9fd4-c8d106a1e0e6,46.39.54.3,2017-06-30 23:52:14.948,https://www.sonarworks.com/speakers?utm_source...,False,Russia
38,d8e3a64a-0db4-43a7-904d-63065210589f,83.184.238.8,2017-06-30 23:52:00.812,https://www.sonarworks.com/speakers?utm_source...,False,Sweden


In [15]:
# Viss kartiba
# Izveidosim jaunu pandas.Series objektu - result, kas satures kolonas ar valsts 
# nosaukumu un unikalu lietotaju skaitu no tas
# Aiznem 64 ms
result = df.country.value_counts()
result.head()

United States     25679
Germany           12683
United Kingdom    10702
Russia             7133
Italy              5747
Name: country, dtype: int64

In [16]:
# Viss ok
# Pielabosim indeksa/kolonas vardus un ierakstisim faila 'result.xlsx'
# Aiznem 44 ms
result = result.rename('User Count')
writer = pd.ExcelWriter('result.xlsx')
result.to_excel(writer,'result.xlsx', index_label='Country')
writer.save()
# Tas ari viss. Tagad mape atrodas fails result.xlsx, kur ir saskaititi visi unikalie lietotaji pa valstim, 
# ņemot vera valsti, no kuras viņi pirmo reizi apmeklejusi web
# Laika zina visas darbibas aiznem ~4-5 min uz Intel Celeron 1000M(1,8ghz duo, 2 MB cache)